In [21]:
#Miles' packages
from astropy.table import Table
import astropy.coordinates as coord
import glob
from astropy import units as u
from astropy.coordinates import SkyCoord

#data processing
import pandas as pd
import numpy as np
import pickle
import math as math
from tqdm import tqdm
import collections

#visualizations
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('Agg')
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

#models
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#data manipulation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

#for my sanity
import warnings
warnings.filterwarnings('ignore')

In [2]:
from astroquery.gaia import Gaia

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [3]:
stream_data_base = 'gaia_mock_streams/'
stream_files = glob.glob(stream_data_base + 'stream*.fits.gz')

We now have all the stream files:

In [40]:
print(len(stream_files))
new_streams = stream_files[150:170]
new_streams = [i[18:-8] for i in new_streams]

322


In [41]:
new_streams

['stream-2243',
 'stream-1090',
 'stream-475',
 'stream-1052',
 'stream-5323',
 'stream-810',
 'stream-390',
 'stream-9211',
 'stream-9971',
 'stream-2165',
 'stream-4945',
 'stream-7736',
 'stream-8754',
 'stream-9060',
 'stream-2927',
 'stream-1727',
 'stream-2835',
 'stream-7595',
 'stream-8290',
 'stream-3438']

In [7]:
new_dfs = []

for i in new_streams:
    table = Table.read(i, format='fits')
    stream = table.to_pandas()
    new_dfs.append(stream)

In [13]:
count_under2000 = 0
for i in new_dfs:
    if len(i) <= 2000:
        count_under2000 += 1
    
count_under2000

10

In [24]:
for df in new_dfs:
    c = SkyCoord(ra=df['ra']*u.degree, dec=df['dec']*u.degree, frame='icrs')
    b = [i.deg for i in c.galactic.b]
    df['b']=b
    df['meets_b_req'] = (abs(df['b']) > 30)
    valid_b = df[abs(df.b)>30]
    rand_bool = np.random.choice(a=[True, False], size=(len(table),),p = [0.1,0.9])
    table['known_to_model'] = rand_bool
    #plt.scatter(new_dfs[i].ra, new_dfs[i].dec)

In [29]:
valid_b = []
for i in new_dfs:
    len_valid_b = i.meets_b_req.sum()
    valid_b.append(len_valid_b)

In [31]:
new_dfs[0].head()

,ra,dec,parallax,pmra,pmdec,radial_velocity,parallax_error,pmra_error,pmdec_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,b,meets_b_req
0,323.640863,-11.699268,-1.176471,-3.557932,0.918430,NaN,0.837489,1.286397,0.961693,19.598420,19.852948,19.090178,-41.429883,True
1,109.044477,-49.128250,0.072034,0.841332,-1.205142,NaN,0.452615,0.721337,0.732172,19.419039,19.960183,18.726932,-16.383134,False
2,59.689032,-55.872726,-0.629604,-0.134809,-0.123856,NaN,0.442995,0.747127,0.761106,19.430418,19.831858,18.887974,-45.978221,True
3,314.568470,-2.628807,0.086998,-2.971311,0.202512,NaN,0.345018,0.513207,0.400734,18.455558,18.847957,17.873999,-29.222595,False
4,42.608531,-55.295570,0.417939,0.780969,-0.160572,NaN,0.464519,0.740308,0.751428,19.459889,19.835722,18.879944,-54.566823,True


In [53]:
stream_name = []
ra_min = []
ra_max = []
dec_min = []
dec_max = []

for item in tqdm(range(len(new_dfs))):
    if valid_b[item]>20:
        stream_name.append(new_streams[item])
        j = new_dfs[item]
        stream_valid_b = j[j.meets_b_req == True]
        ra_min.append(min(stream_valid_b.ra))
        ra_max.append(max(stream_valid_b.ra))
        dec_min.append(min(stream_valid_b.dec))
        dec_max.append(max(stream_valid_b.dec))
    
cut_untouched = pd.DataFrame()
cut_untouched['stream_file']= stream_name
cut_untouched['ra_min']= ra_min
cut_untouched['ra_max']= ra_max
cut_untouched['dec_min']= dec_min
cut_untouched['dec_max']= dec_max

cut_untouched.to_csv("cuts/cuts9.csv", index = False)

100%|██████████| 20/20 [00:00<00:00, 606.67it/s]


In [54]:
cut_untouched

,stream_file,ra_min,ra_max,dec_min,dec_max
0,stream-2243,0.609968,359.861833,-75.580219,34.010314
1,stream-475,0.055743,359.350627,-85.565780,77.956716
2,stream-1052,204.351490,335.380115,-69.240843,44.932758
3,stream-5323,0.071507,359.420596,-47.867433,62.141058
4,stream-810,12.845470,350.417566,-82.492558,67.448588
5,stream-390,1.446175,358.237114,-68.547907,78.713001
6,stream-9211,0.006799,359.414626,-62.238577,14.616133
7,stream-9971,0.040862,359.946139,-77.847182,65.838253
8,stream-2165,0.356971,359.630481,-42.714888,78.148066
9,stream-4945,194.132145,203.755177,-12.420594,0.638192


In [46]:
cut_untouched

,stream_file,ra_min,ra_max,dec_min,dec_max
0,stream-2243,0.609968,359.861833,-75.580219,34.010314
1,stream-475,0.055743,359.350627,-87.148499,77.956716
2,stream-1052,194.693033,335.380115,-78.851299,44.932758
3,stream-5323,0.071507,359.420596,-48.171601,62.141058
4,stream-810,12.845470,350.417566,-85.569549,69.625400
5,stream-390,1.446175,358.237114,-68.547907,81.272719
6,stream-9211,0.006799,359.414626,-66.746894,21.041951
7,stream-9971,0.040862,359.946139,-78.402552,69.077757
8,stream-2165,0.015023,359.630481,-46.250169,81.510166
9,stream-4945,194.132145,203.755177,-12.420594,0.638192


In [14]:
training = pd.read_csv('stream_stars_split/group_a/group_a_counts_stars_more_than_fifty.csv')
streams_a = list(training.stream_name)

group_b = 'stream_stars_split/group_b/'
group_b_known = glob.glob(group_b + 'known_to_model/' + '*.csv')
group_b_known = [i[50:-19] for i in group_b_known]

group_c = 'stream_stars_split/group_c/'
group_c_known = glob.glob(group_c + 'known_to_model/' + '*.csv')
group_c_known = [i[50:-19] for i in group_c_known]

val_test = group_b_known #+ group_c_known

In [15]:
group_b_limits = pickle.load( open( "new_stream_cut_limits_group_b.pkl", "rb" ) ).T.reset_index()
group_b_limits = group_b_limits.drop(['index'], axis = 1)
group_c_limits = pickle.load( open( "new_stream_cut_limits_group_c.pkl", "rb" ) ).T.reset_index()
group_c_limits = group_c_limits.drop(['index'], axis = 1)

In [16]:
intelligent_cuts_loc = []
for i in streams_a:
    intelligent_cuts_loc.append('stream_stars_split/group_a/'+i+'_intelligent_cut.csv')
for i in group_b_known:
    intelligent_cuts_loc.append('stream_stars_split/group_b_intelligent/'+i+'_intelligent_cut.csv')
for i in group_c_known:
    intelligent_cuts_loc.append('stream_stars_split/group_c_intelligent/'+i+'_intelligent_cut.csv')

## Filter by abs(b) and write to 'csv'

In [ ]:
#filter group_b
stream_b_names = list(group_b_limits.stream_name)
stream_b_names = stream_b_names[:3] + stream_b_names[4:]
stream_stars_b_known = ['stream_stars_split/group_b/known_to_model_intelligent/' + i + '_intelligent_cut.csv' for i in stream_b_names]
stream_stars_b_unknown= ['stream_stars_split/group_b/unknown_to_model_intelligent/' + i + '_intelligent_cut.csv' for i in stream_b_names]

for item in tqdm(range(len(stream_stars_b))): #range(len(stream_stars))
    table_known = pd.read_csv(stream_stars_b_known[item], index_col = 0)
    table_unknown = pd.read_csv(stream_stars_b_unknown[item], index_col = 0)
    table = table_known.append(table_unknown)
    c = SkyCoord(ra=table['ra']*u.degree, dec=table['dec']*u.degree, frame='icrs')
    b = [i.deg for i in c.galactic.b]
    table['b']=b
    table['meets_b_req'] = (abs(table['b']) > 30)
    valid_b = table[abs(table.b)>30]
    rand_bool = np.random.choice(a=[True, False], size=(len(table),),p = [0.1,0.9])
    table['known_to_model'] = rand_bool
    table.to_csv('stream_stars_split/group_b_intelligent/'+stream_b_names[item]+ '_intelligent_cut.csv')

In [34]:
original_lengths = []
b_is_valid_lengths = []
model_lengths = []

for i in tqdm(range(len(intelligent_cuts_loc))):
    table = pd.read_csv(intelligent_cuts_loc[i])
    original_lengths.append(len(table))
    table_b = table[abs(table.b)>30]
    b_is_valid_lengths.append(len(table_b))
    if i>34:
        table_model = table_b[table_b.known_to_model == True]
        model_lengths.append(len(table_model))
    else:
        model_lengths.append(len(table_b))
    

100%|██████████| 53/53 [00:00<00:00, 216.28it/s]


In [41]:
stream_name_2 = [i[27:-20] for i in intelligent_cuts_loc[:35]]
stream_name_3 = [i[39:-20] for i in intelligent_cuts_loc[35:]]
stream_name_4 = stream_name_2 + stream_name_3
comp = pd.DataFrame()
comp['stream_name']=stream_name_4
comp['original_lengths']= original_lengths
comp['b_valid']=b_is_valid_lengths
comp['model_lengths']= model_lengths

min_model = 2
comp_filt = comp[comp.model_lengths > min_model]
comp_filt


,stream_name,original_lengths,b_valid,model_lengths
0,stream-5711,63,63,63
1,stream-4106,627,490,490
2,stream-522,607,3,3
3,stream-6446,275,257,257
4,stream-4468,302,302,302
5,stream-847,216,191,191
6,stream-5982,240,154,154
9,stream-5830,166,164,164
11,stream-4990,261,199,199
12,stream-1864,302,297,297


In [193]:
comp.to_csv('valid_b.csv')

In [194]:
len(comp[comp.b_valid<60])

14

## Brian doesn't need below this location

In [46]:
#training = pd.read_csv('stream_stars_split/group_a/group_a_counts_stars_more_than_fifty.csv')
streams_adhoc = list(comp_filt.stream_name)
#streams_adhoc = val_test

stream_files_adhoc = ['gaia_mock_streams/' + i + '.fits.gz' for i in streams_adhoc]


#stream_files_adhoc = ['gaia_mock_streams/stream-5711.fits.gz','gaia_mock_streams/stream-4468.fits.gz']
#stream_files_adhoc += ['gaia_mock_streams/stream-3526.fits.gz','gaia_mock_streams/stream-4682.fits.gz']
#stream_files_adhoc += ['gaia_mock_streams/stream-1954.fits.gz','gaia_mock_streams/stream-9420.fits.gz']
#stream_files_adhoc += ['gaia_mock_streams/stream-9164.fits.gz']

In [133]:
print(stream_files_adhoc[0])
print(list(comp_filt.stream_name)[0])
relevant_inx = list(comp_filt[comp_filt.model_lengths>2].model_lengths.index)
intelligent_cuts_loc_meets_cut = [intelligent_cuts_loc[i] for i in range(len(intelligent_cuts_loc)) if i in relevant_inx]
b_is_valid_meets_cut = [b_is_valid_lengths[i] for i in range(len(intelligent_cuts_loc)) if i in relevant_inx]

gaia_mock_streams/stream-5711.fits.gz
stream-5711


In [75]:
len(intelligent_cuts_loc_meets_cut), len(relevant_inx), len(comp_filt)

(46, 46, 46)

In [134]:
len(b_is_valid_meets_cut)

46

#### Let us generate a CSV with new ra_min, ra_max, dec_min, and dec_max

In [76]:
stream_name = []
ra_min = []
ra_max = []
dec_min = []
dec_max = []

for item in tqdm(range(len(comp_filt))):
    stream = pd.read_csv(intelligent_cuts_loc_meets_cut[item])
    
    stream_name.append(list(comp_filt.stream_name)[item])
    ra_min.append(min(stream.ra))
    ra_max.append(max(stream.ra))
    dec_min.append(min(stream.dec))
    dec_max.append(max(stream.dec))
    
cut_untouched = pd.DataFrame()
cut_untouched['stream_file']= stream_name
cut_untouched['ra_min']= ra_min
cut_untouched['ra_max']= ra_max
cut_untouched['dec_min']= dec_min
cut_untouched['dec_max']= dec_max

cut_untouched.to_csv("cuts/cuts8.csv", index = False)

100%|██████████| 46/46 [00:00<00:00, 259.52it/s]


In [78]:
#relevant cut for each stream
#cuts1 = pd.read_csv("cuts/cuts.csv")
#cuts2 = pd.read_csv("cuts/cuts2.csv")
#cuts = cuts1.append(cuts2, ignore_index = True, sort = True)

#untouched cuts (validation / performance reporting)
cuts_1 = pd.read_csv("cuts/cuts8.csv")
cuts = cuts_1


# group_b_limits = pickle.load( open( "new_stream_cut_limits_group_b.pkl", "rb" ) ).T.reset_index()
# group_c_limits = pickle.load( open( "new_stream_cut_limits_group_c.pkl", "rb" ) ).T.reset_index()
# cuts = group_b_limits.append(group_c_limits, ignore_index = True, sort = False)
# cuts = cuts[cuts.stream_name != 'stream-6982']

# #cuts = pickle.load( open( "new_stream_cut_limits.pkl", "rb" ) ).T.reset_index()
# cuts.stream_name = 'gaia_mock_streams/' + cuts.stream_name + '.fits.gz'
# cuts = cuts.rename(columns={"stream_name": "stream_file"})
# cuts = cuts.drop(['index'], axis = 1)

In [148]:
cuts.head()
stream_names_write = list(cuts.stream_file)

#### let us write a function to extract the Gaia noise points based on some inputs

In [89]:
def obtain_noise(min_ra, max_ra, min_dec, max_dec, max_rel_err, b_threshold, n_points):
    
    qry = f" \n\
    select top {n_points} source_id, \n\
    dr2.ra, \n\
    dr2.dec, \n\
    parallax, \n\
    parallax_error, \n\
    pmra, \n\
    pmdec, \n\
    b, \n\
    phot_g_mean_mag,\n\
    phot_bp_mean_mag, \n\
    phot_rp_mean_mag, \n\
    bp_rp, \n\
    bp_g, \n\
    g_rp\n\
    from gaiadr2.gaia_source as dr2 \n\
    where dr2.ra > {min_ra} and dr2.ra < {max_ra} and dr2.dec > {min_dec} and dr2.dec < {max_dec} \n\
    and parallax is not null \n\
    and parallax_error is not null \n\
    and abs(dr2.parallax/dr2.parallax_error) < {max_rel_err} \n\
    and pmra is not null \n\
    and pmdec is not null \n\
    and phot_g_mean_mag is not null \n\
    and phot_bp_mean_mag is not null \n\
    and phot_rp_mean_mag is not null \n\
    and bp_rp is not null \n\
    and bp_g is not null \n\
    and g_rp is not null \n\
    and abs(b) > {b_threshold} \n\
    order by random_index"

    data_noise = Gaia.launch_job_async(qry).get_results().to_pandas()
    
    return data_noise

#### For each mock stream, we wish to obtain a fixed test set that represents the realistic ratio of non-stream stars to stream stars that we expect after applying an isochrone filter.
#### - This ratio was previously determined to be ~400

In [190]:
def obtain_test_set(list_of_stellar_streams, multiple):
    y=40
    b_threshold = 30
    for item in tqdm(range(len(list_of_stellar_streams))):
#         table = Table.read(list_of_stellar_streams[item], format='fits')
#         stream = table.to_pandas()
        
        idx = cuts.index[cuts.stream_file==list(cuts.stream_file)[item+y]][0]

        ra_min = cuts.loc[idx].ra_min
        ra_max = cuts.loc[idx].ra_max
        dec_min = cuts.loc[idx].dec_min
        dec_max = cuts.loc[idx].dec_max
        
        #restrict stream to relevant portion
        #stream = stream.query('ra > ' + str(ra_min) + ' & ra < ' + str(ra_max) + ' & dec > ' + str(dec_min) + ' & dec < ' + str(dec_max))
        stream_length = b_is_valid_meets_cut[item+y]
        
        
        #obtain noise points such that:
        #- the ratio of stream to noise points in the test set is 1:multiple
        #- this is required because KNN introduces bias when the ratios are imbalanced
        n_points = stream_length * multiple
        #use max_rel_err of 0.5
        max_rel_err =  0.5
        #we now select our noise points that we will incorporate into our training and test set
        noise_points = obtain_noise(ra_min, ra_max, dec_min, dec_max, max_rel_err, b_threshold, n_points)

        #label our data as "not part of the stream"
        noise_points['stream_mask'] = False

        #send to csv
        name = stream_names_write[item+y]+'_mul_400_total_noise.csv'
        noise_points.to_csv(name)
        print(item, idx, ra_min, ra_max, stream_length, n_points, len(noise_points), name)

    return noise_points

In [8]:
####ONLY NEED TO RUN THIS ONCE
#viable_streams = stream_files_adhoc
#test2 = obtain_test_set(viable_streams[40:41],400)